In [1]:
# type stability is the idea that the type of the return value of a function is dependent only on the type of its arguments and not the values

function pos(x)
    if x<0
        return 0
    else
        return x
    end
    
end


pos (generic function with 1 method)

In [4]:
@show typeof(pos(-1))
@show typeof(pos(-2.5))



typeof(pos(-1)) = Int64
typeof(pos(-2.5)) = Int64


Int64

In [6]:
function pos_fixed(x)
    if x>0
        return zero(x)
    else
        return x
    end    
end
pos_fixed(2.5)

0.0

In [9]:
using BenchmarkTools
@btime pos(2.5)
@btime pos_fixed(2.5)

  0.001 ns (0 allocations: 0 bytes)
  0.001 ns (0 allocations: 0 bytes)


0.0

In [11]:
@code_warntype pos(2.5)

Variables
  #self#::Core.Compiler.Const(pos, false)
  x::Float64

Body::Union{Float64, Int64}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x


In [14]:
@code_warntype pos(1)


Variables
  #self#::Core.Compiler.Const(pos, false)
  x::Int64

Body::Int64
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x


In [15]:
@code_llvm pos(2.5)


;  @ In[1]:3 within `pos'
; Function Attrs: uwtable
define { %jl_value_t*, i8 } @julia_pos_1832([8 x i8]* noalias nocapture align 8 dereferenceable(8), double) #0 {
top:
;  @ In[1]:4 within `pos'
; ┌ @ float.jl:503 within `<' @ float.jl:458
   %2 = fcmp uge double %1, 0.000000e+00
; └
  br i1 %2, label %L12, label %L11

L11:                                              ; preds = %L12, %top
  %merge = phi { %jl_value_t*, i8 } [ { %jl_value_t* inttoptr (i64 396591136 to %jl_value_t*), i8 -126 }, %top ], [ { %jl_value_t* null, i8 1 }, %L12 ]
;  @ In[1]:5 within `pos'
  ret { %jl_value_t*, i8 } %merge

L12:                                              ; preds = %top
;  @ In[1]:7 within `pos'
  %.0..sroa_cast = bitcast [8 x i8]* %0 to double*
  store double %1, double* %.0..sroa_cast, align 8
  br label %L11
}


In [17]:
@code_llvm pos_fixed(2.5)


;  @ In[6]:1 within `pos_fixed'
; Function Attrs: uwtable
define double @julia_pos_fixed_1838(double) #0 {
top:
;  @ In[6]:2 within `pos_fixed'
  %.inv = fcmp ogt double %0, 0.000000e+00
  %spec.select = select i1 %.inv, double 0.000000e+00, double %0
;  @ In[6]:3 within `pos_fixed'
  ret double %spec.select
}


In [21]:
@show typeof(sqrt(5))

typeof(sqrt(5)) = Float64


Float64

In [23]:
#varables inside a for-loop should be type-stable to get high performance
function sum_sqrt(n)
    r=0
    for i in 1:n
        r=r+sqrt(i)
    end
    return r
end

@code_warntype sum_sqrt(10)


Variables
  #self#::Core.Compiler.Const(sum_sqrt, false)
  n::Int64
  r::Union{Float64, Int64}
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Union{Float64, Int64}
1 ─       (r = 0)
│   %2  = (1:n)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = r::Union{Float64, Int64}
│   %11 = Main.sqrt(i)::Float64
│         (r = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return r
